In [211]:
import pandas as pd
import datetime as dt
import numpy as np
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import unicodedata
import psycopg2
import time
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import string
import datetime

In [212]:
hosturl = "ec2-54-221-107-226.compute-1.amazonaws.com"

In [227]:
#gather tweets
conn = psycopg2.connect(database="postgres", user="power_user", password="$poweruserpassword", host=hosturl, port="5432")
cur = conn.cursor()

query_string = "SELECT ID, TEXT, UPDATE_TIME, NOUNS, SENTIMENT from tweets_analytics"
query_string += " WHERE CAST(UPDATE_TIME AS TIMESTAMP) >  '2017-12-19 21:00:00'"
query_string += " and CAST(UPDATE_TIME AS TIMESTAMP) < '2017-12-19 22:00:00'"
# utc_datetime = datetime.datetime.utcnow()
# currTimeUTC = utc_datetime.strftime("%Y-%m-%d %H:%M:%S")
# query_string += "2017-12-19 22:00:00"
print(query_string)
cur.execute(query_string)

tweets = cur.fetchall()
conn.close()
cols = ["ID", "Text", "Time", "Nouns", "Sentiment"]
tweets = pd.DataFrame(tweets, columns = cols)

SELECT ID, TEXT, UPDATE_TIME, NOUNS, SENTIMENT from tweets_analytics WHERE CAST(UPDATE_TIME AS TIMESTAMP) >  '2017-12-19 21:00:00' and CAST(UPDATE_TIME AS TIMESTAMP) < '2017-12-19 22:00:00'


In [229]:
tweets.describe()

,ID,Text,Time,Nouns,Sentiment
count,1303,1303,1303,1303,1303
unique,1303,1300,348,1169,327
top,943228697234083840,予備校時代、英作文の問題で「覆水盆にかえらず」を「Mr.Fukusui did not go...,2017-12-19 21:00:10,,0.0
freq,1,3,15,34,584


In [230]:
tweets.head()

,ID,Text,Time,Nouns,Sentiment
0,943237863885459457,I need a tropical vacation ASAP,2017-12-19 21:53:30,ASAP,0.0
1,943237871833554944,#WeMissYouMaineDay23 https://t.co/4ujpJgutbD,2017-12-19 21:53:30,WeMissYouMaineDay23,0.0
2,943225262409334784,Vacation In Hell : shot by me https://t.co/dnF...,2017-12-19 21:03:30,"Vacation,Hell,shot",-0.6808
3,943226134526939136,@linuspoint @RuthDavidsonMSP Brexit is the equ...,2017-12-19 21:06:50,"@linuspoint,@RuthDavidsonMSP,Brexit,cost,famil...",0.0
4,943224419245228033,@ExtendedStay @Chandlermom221 Where are you go...,2017-12-19 21:00:10,"@ExtendedStay,@Chandlermom221,Where,year,,Sant...",0.0


In [231]:
#gather cities
conn = psycopg2.connect(database="postgres", user="power_user", password="$poweruserpassword", host=hosturl, port="5432")
cur = conn.cursor()
cur.execute("SELECT * from worldcities")

#cities = pd.read_csv("cities.csv")
cities = cur.fetchall()
conn.close()
cols = ["city", "city_ascii", "lat", "long", "pop", "country", "iso2", "iso3", "province"]
cities = pd.DataFrame(cities, columns = cols)

In [232]:
cities.describe()
cities['lat'] = pd.to_numeric(cities['lat'])
cities['long'] = pd.to_numeric(cities['long'])
for c in cities.columns:
    print(cities[c].dtype)

object
object
float64
float64
object
object
object
object
object


In [233]:
#Now form a df, given a city, we get its lat/long
df_city=cities.set_index("city_ascii")

#Now form a df, given a country, we get its averaged lat/long
countryLats = cities.groupby(['country'])['lat'].mean()
countryLongs = cities.groupby(['country'])['long'].mean()
df_country = pd.DataFrame(countryLats, columns=['lat'])
df_country['long'] = countryLongs
df_province.head(10)

#Now form a df, given a province, we get its averaged lat/long
provinceLats = cities.groupby(['province'])['lat'].mean()
provinceLongs = cities.groupby(['province'])['long'].mean()
df_province = pd.DataFrame(provinceLats, columns=['lat'])
df_province['long'] = provinceLongs
df_province.head(10)



lat,long = df_city.loc["Madrid" , ["lat", "long"]]
print(lat,long)

lat,long = df_country.loc["Spain" , ["lat", "long"]]
print(lat,long)

lat,long = df_province.loc["Castilla-La Mancha" , ["lat", "long"]]
print(lat,long)

40.40002626 -3.683351686
39.231296892 -4.26383970298
39.83369563 -3.017767851


In [234]:
city_list = cities["city_ascii"].tolist()
city_set = set(city_list)
city_list = list(city_set)

country_list = cities["country"].tolist()
country_set = set(country_list)
country_list = list(country_set)

province_list = cities["province"].tolist()
province_set = set(province_list)
province_list = list(province_set)

a = province_set.union(city_set)
locations = a.union(country_set)
locations = list(locations)
locations = locations[1:]
print(len(locations))

8944


In [235]:
def getPrintableString(inputStr):
    printable = set(string.printable)
    outputStr = list(filter(lambda x: x in printable, inputStr))
    outputStr = "".join(outputStr)
    return outputStr

In [236]:
def matchPlaces(text, countryList, provinceList, cityList):
    matches = []
    for word in text.split():
        place,lat,long = (None,None,None)
        match = process.extractOne(word, countryList, scorer=fuzz.ratio)
        if match[1] > 95:
            country = match[0]
            lat,long = df_country.loc[country , ["lat", "long"]]
            if isinstance(lat, float) and isinstance(long, float):
                matches.append((country,lat,long))
        else:
            match = process.extractOne(word, provinceList, scorer=fuzz.ratio)
            if match[1] > 95:
                province = match[0]
                lat,long = df_province.loc[province , ["lat", "long"]]
                if isinstance(lat, float) and isinstance(long, float):
                    matches.append((province,lat,long))
            else:
                match = process.extractOne(word, cityList, scorer=fuzz.ratio)
                if match[1] > 95:
                    city = match[0]
                    lat,long = df_city.loc[city , ["lat", "long"]]
                    if isinstance(lat, float) and isinstance(long, float):
                        matches.append((city,lat,long))
                
                
    return matches

In [237]:
#method to insert data
def insertdata(data):
    conn = psycopg2.connect(database="postgres", user="power_user", password="$poweruserpassword", host=hosturl, port="5432")
    cur = conn.cursor()
    cur.execute("INSERT INTO location_analytics (TEXT,ID, UPDATE_TIME, SENTIMENT, PLACE, LATITUDE, LONGITUDE) \
          VALUES (%s,%s,%s,%s,%s,%s,%s)",(data[0],data[1],data[2],data[3],data[4],data[5],data[6]))
    conn.commit()
    conn.close()

In [238]:
start_time = time.time()
cols = ["ID", "Text", "Time", "Nouns", "Sentiment"]
insert_count = 0
tweet_count = 0
print("Total tweets to process: {0}".format(tweets.shape[0]))
for i in range(tweets.shape[0]):
    tweet = tweets.loc[i]
    tweet_count += 1
    nouns = getPrintableString(tweet["Nouns"])
    nouns = nouns.replace(",", " ")
    places = matchPlaces(nouns, country_list, province_list, city_list)
    for p in range(len(places)):
        data = []
#         print("Nouns={0}\nMatched Place={1}".format(nouns, places))
        data.append(tweet["Text"])
        data.append(tweet["ID"])
        data.append(tweet["Time"])
        data.append(tweet["Sentiment"])
        data.append(places[p][0]) #place
        data.append(places[p][1]) #lat
        data.append(places[p][2]) #long
        print(data)
        insertdata(data)
        insert_count += 1
        print("Insert count =  {0}".format(insert_count))

print("Processed {0} tweets in {1} seconds".format(tweet_count, time.time() - start_time))

Total tweets to process: 1303


['His and Her drinks by the pool on this December afternoon. A real Floridian holiday vacation! 🌴  || Photo: maragon3… https://t.co/OTrKAEJOHl', '943224435586359297', '2017-12-19 21:00:10', '0.4574', 'Pool', -4.3075928344999994, 15.023344049999999]
Insert count =  1


['@aira_fondon I’m sorry to inform you that the rest of your vacation is cancelled. It’s time for you to come back to Ohio.', '943224544290136066', '2017-12-19 21:00:40', '-0.3182', 'Ohio', 40.532618761428573, -82.797219598571431]
Insert count =  2


['Orlando family-friendly vacation rental! All the gear! &lt; 2 miles to #Disney! https://t.co/rLrXfjgQch #familytravel https://t.co/uQiptmKFxA', '943224595871649801', '2017-12-19 21:00:50', '0.0', 'Orlando', 28.509976829999999, -81.380030360000006]
Insert count =  3
['NCIS or Sanford and Son marathon. On vacation facing hard decisions.', '943224619779198976', '2017-12-19 21:00:50', '-0.1027', 'Sanford', 28.78995974, -81.279984780000007]
Insert count =  4
['NCIS or Sanford and Son marathon. On vacation facing hard decisions.', '943224619779198976', '2017-12-19 21:00:50', '-0.1027', 'Marathon', 48.75039512, -86.366651039999994]
Insert count =  5
['Vacation in Hawaii: https://t.co/xegZFMozOn via @YouTube', '943224697600307200', '2017-12-19 21:01:10', '0.0', 'Hawaii', 20.837251564999999, -157.14048821666668]
Insert count =  6
['Chris Martin and Dakota Johnson vacation in Paris – Wonderwall - Chris Martin and Dakota Johnson vacation in Paris… https://t.co/bcdqlkGaJH', '943224707284955136',

['Second vacation day in Cozumel. Playing with the birds 🐦🏝 @ Cozumel, Mexico https://t.co/K3T9BRVuAi', '943224826398953472', '2017-12-19 21:01:40', '0.2023', 'Cozumel', 20.51000002, -86.950000450000005]
Insert count =  16
['Second vacation day in Cozumel. Playing with the birds 🐦🏝 @ Cozumel, Mexico https://t.co/K3T9BRVuAi', '943224826398953472', '2017-12-19 21:01:40', '0.2023', 'Cozumel', 20.51000002, -86.950000450000005]
Insert count =  17
['Second vacation day in Cozumel. Playing with the birds 🐦🏝 @ Cozumel, Mexico https://t.co/K3T9BRVuAi', '943224826398953472', '2017-12-19 21:01:40', '0.2023', 'Mexico', 22.496705073280435, -101.12578106915348]
Insert count =  18
['Vacation house goals 🌺 #wanderlust #greece https://t.co/5pHNyldewH', '943224968812273664', '2017-12-19 21:02:20', '0.0', 'Greece', 38.617972865666665, 23.749089734999998]
Insert count =  19
['Roma . ❤️ #birthday\n.\n.\n. \n#travel #traveling #italy #roma #vacation #visiting #instatravel… https://t.co/IZSSyoiAlv', '9432249

...

['@girlof65roses @mibellaamor @jordynmccrazy12 @TylerBaltierra @CatelynnLowell @MTV @TMZ If Brandon and Teresa didn’t… https://t.co/le5C7w9KoY', '943236983089807361', '2017-12-19 21:50:00', '0.0', 'Brandon', 49.833274760000002, -99.949982140000003]
Insert count =  196
['Ormara Beach, Balochistan, Pakistan #Travel #Nature #Holiday #Vacation #Journey #Beautiful #EmergingPakistan… https://t.co/3o56VNbhK7', '943236995643531264', '2017-12-19 21:50:10', '0.0', 'Pakistan', 30.88725971536585, 71.047199973170763]
Insert count =  197
['Hotels in Hong Kong\n \n➡ https://t.co/cdWEY97suH\n \n#HongKong #China #Traveller #Travel #Vacation #Destination #Hotel #Holiday #HotelDeals', '943237051729588224', '2017-12-19 21:50:20', '0.0', 'China', 35.389668289700005, 114.002471625175]
Insert count =  198
['The Jas #Villas\n\n&gt;https://t.co/5q6xxusaBx\n\n#Bali #BookingHotel #Travel #Traveller #Holiday #HotelDeals #Vacation https://t.co/xzqZ2RPfiv', '943237076484366336', '2017-12-19 21:50:20', '0.0', 'Bali'

['At least 11 killed, 18 injured in Mexico tour bus crash that was a part of a cruise ship vacation - ABC News -… https://t.co/DTH0gWWsCg', '943237235117260800', '2017-12-19 21:51:00', '-0.872', 'Mexico', 22.496705073280435, -101.12578106915348]
Insert count =  200
['Reposting @yuvanta:\nCappadocia, 🇹🇷 📸 by @kyrenian\n.\n.\n.\n.\n.\n#Cappadocia #Turkey #Sunset #Scenic #Holiday #Vacation… https://t.co/qilxLlI3e5', '943237297008410625', '2017-12-19 21:51:20', '0.0', 'Turkey', 39.124500993571417, 34.841018835476191]
Insert count =  201
['Time to start planning YOUR Florida Vacation !\nPlease enjoy our video and give us a call for Amazing homes at Fanta… https://t.co/lVXtQ1M7S6', '943237308140179457', '2017-12-19 21:51:20', '0.8622', 'Florida', 26.504501297948725, -81.285591428974357]
Insert count =  202


['Merida Mexico: Ultimate Travel Guide. Discover the Food, History &amp; Culture of this beautiful Mexican City… https://t.co/J7VrG7cibq', '943237549618757632', '2017-12-19 21:52:20', '0.6833', 'Mexico', 22.496705073280435, -101.12578106915348]
Insert count =  203
['Merida Mexico: Ultimate Travel Guide. Discover the Food, History &amp; Culture of this beautiful Mexican City… https://t.co/J7VrG7cibq', '943237549618757632', '2017-12-19 21:52:20', '0.6833', 'Guide', 36.045098289999999, 101.42418620000001]
Insert count =  204
['I can’t go on a vacation so i’m spending all of next week in houston.', '943237607521210369', '2017-12-19 21:52:30', '0.0', 'Houston', 29.819974380000001, -95.339979290000002]
Insert count =  205
['Well, hey, it’d be a good excuse for me to go on vacation. To Egypt.\n\n*I smirk, letting out a laugh.* https://t.co/X7Y7Rb3Uv0', '943237705810546688', '2017-12-19 21:52:50', '0.6486', 'Egypt', 28.758547533333335, 31.095179538571436]
Insert count =  206
['Hotels in Austra

['Need this vacation man. Not even going anywhere 😂😂😂', '943238434726010886', '2017-12-19 21:55:50', '0.0', 'Man', 7.4004126170000006, -7.5499890560000003]
Insert count =  216
['Back to work Tuesday after a quick trip to the East Coast, what a vacation!', '943238606264643584', '2017-12-19 21:56:30', '0.0', 'Coast', -3.1219494695714287, 39.718038132857146]
Insert count =  217
['In a water lock, Russia https://t.co/14LnH1rQBo #travel #paradise #vacation https://t.co/3V71Tky1j8', '943238637663281152', '2017-12-19 21:56:40', '0.0', 'Russia', 56.643202495751311, 73.978937705094978]
Insert count =  218


['Planning Your Mexican Vacation - Travel Documents Needed For Mexico Trips https://t.co/N54ZW5QSTt https://t.co/hPm4OjN0xR', '943238826113228800', '2017-12-19 21:57:20', '0.0', 'Mexico', 22.496705073280435, -101.12578106915348]
Insert count =  219
['I wanna go on a vacation, i haven’t been somewhere outside of Michigan in sooooo long. 😩', '943238846271119361', '2017-12-19 21:57:30', '0.0', 'Michigan', 44.119044032727274, -85.470358560000008]
Insert count =  220
['Cappadocia, #Turkey\nIs it on your list of #Vacation list?  It should be!\nhttps://t.co/e9EOx50Pv6 https://t.co/IgxdAzf42E', '943239066828529665', '2017-12-19 21:58:20', '0.0', 'Turkey', 39.124500993571417, 34.841018835476191]
Insert count =  221
Processed 1303 tweets in 366.6546514034271 seconds
